# Kotlin 고급 기능
## 확장 함수, 값 클래스, 위임, 코루틴, DSL

이 노트북에서는 Kotlin의 고급 기능들을 학습합니다.

### 학습 내용
1. 확장 함수 (Extension Functions)
2. 값 클래스 (Value Classes / Inline Classes)
3. 위임 (Delegation)
4. 코루틴 (Coroutines)
5. DSL (Domain Specific Language)

## 1. 확장 함수 (Extension Functions)

기존 클래스에 새로운 메서드를 추가하는 것처럼 보이게 하는 기능입니다. C#의 확장 메서드와 유사합니다.

In [ ]:
// String 클래스에 확장 함수 추가
fun String.removeSpaces(): String = this.replace(" ", "")

val text = "Hello Kotlin World"
println("원본: '$text'")
println("공백 제거: '${text.removeSpaces()}'")

// this는 수신 객체를 가리킴
fun String.addExclamation() = "$this!"
println("느낌표 추가: '${text.addExclamation()}'")

In [ ]:
// 회문 검사 확장 함수
fun String.isPalindrome(): Boolean {
    val cleaned = this.lowercase().filter { it.isLetterOrDigit() }
    return cleaned == cleaned.reversed()
}

// 테스트
val testStrings = listOf(
    "level",
    "A man a plan a canal Panama",
    "race a car",
    "hello"
)

testStrings.forEach { str ->
    println("'$str' -> ${if (str.isPalindrome()) "회문입니다" else "회문이 아닙니다"}")
}

In [ ]:
// 제네릭 확장 함수
fun <T> List<T>.secondOrNull(): T? = if (size >= 2) this[1] else null

fun <T> List<T>.customFilter(predicate: (T) -> Boolean): List<T> {
    val result = mutableListOf<T>()
    for (item in this) {
        if (predicate(item)) {
            result.add(item)
        }
    }
    return result
}

val numbers = listOf(10, 20, 30, 40, 50)
val shortList = listOf(5)

println("두 번째 요소:")
println("$numbers -> ${numbers.secondOrNull()}")
println("$shortList -> ${shortList.secondOrNull()}")

val filtered = numbers.customFilter { it > 25 }
println("\n25보다 큰 수: $filtered")

In [ ]:
// 기본 타입에 대한 확장
fun Int.isEven() = this % 2 == 0
fun Int.isOdd() = !isEven()
fun Int.squared() = this * this
fun Int.factorial(): Long {
    return if (this <= 1) 1L else this * (this - 1).factorial()
}

println("정수 확장 함수:")
println("4.isEven() = ${4.isEven()}")
println("7.isOdd() = ${7.isOdd()}")
println("5.squared() = ${5.squared()}")
println("6.factorial() = ${6.factorial()}")

// 연쇄 호출
val result = 3.squared().squared()
println("\n3의 제곱의 제곱: $result")

In [ ]:
// 확장 함수로 빌더 패턴 구현
class StringBuilder2 {
    private val content = mutableListOf<String>()
    
    fun append(text: String) {
        content.add(text)
    }
    
    override fun toString() = content.joinToString("")
}

// 확장 함수로 체이닝 지원
fun StringBuilder2.appendLine(text: String): StringBuilder2 {
    append("$text\n")
    return this
}

fun StringBuilder2.appendWithSpace(text: String): StringBuilder2 {
    append("$text ")
    return this
}

val builder = StringBuilder2()
    .appendLine("첫 번째 줄")
    .appendLine("두 번째 줄")
    .appendWithSpace("단어1")
    .appendWithSpace("단어2")
    .appendLine("")

println(builder)

### 💡 핵심 포인트
- **기존 클래스를 수정하지 않고** 새 기능 추가
- **수신 객체 타입**에 따라 호출 가능 여부 결정
- **정적으로 해결**되므로 다형성 없음
- **가독성 향상**: `StringUtils.isEmpty(str)` → `str.isEmpty()`

## 2. 값 클래스 (Value Classes / Inline Classes)

런타임 오버헤드 없이 타입 안전성을 제공하는 래퍼 클래스입니다.

In [ ]:
// 값 클래스 정의
@JvmInline
value class Password(val value: String) {
    init {
        require(value.length >= 8) { "비밀번호는 8자 이상이어야 합니다" }
    }
    
    fun isStrong(): Boolean {
        return value.any { it.isDigit() } &&
               value.any { it.isUpperCase() } &&
               value.any { it.isLowerCase() }
    }
}

// 사용
val weakPassword = Password("12345678")
val strongPassword = Password("MyPass123")

println("약한 비밀번호 강도: ${if (weakPassword.isStrong()) "강함" else "약함"}")
println("강한 비밀번호 강도: ${if (strongPassword.isStrong()) "강함" else "약함"}")

// 컴파일 타임에 타입 체크
// val email: Email = weakPassword  // 컴파일 에러!

In [ ]:
// 이메일 값 클래스
@JvmInline
value class Email(val value: String) {
    init {
        require("@" in value && "." in value) { "올바른 이메일 형식이 아닙니다" }
    }
    
    val domain: String
        get() = value.substringAfter("@")
    
    val username: String
        get() = value.substringBefore("@")
}

val email = Email("user@example.com")
println("이메일: ${email.value}")
println("사용자명: ${email.username}")
println("도메인: ${email.domain}")

In [ ]:
// 단위를 표현하는 값 클래스
@JvmInline
value class Celsius(val value: Double) {
    fun toFahrenheit() = value * 9/5 + 32
    fun toKelvin() = value + 273.15
}

@JvmInline
value class Meters(val value: Double) {
    fun toKilometers() = value / 1000
    fun toFeet() = value * 3.28084
}

// 타입 안전한 온도 변환
val temp = Celsius(25.0)
println("온도 변환:")
println("섭씨: ${temp.value}°C")
println("화씨: ${temp.toFahrenheit()}°F")
println("켈빈: ${temp.toKelvin()}K")

// 타입 안전한 거리 변환
val distance = Meters(1500.0)
println("\n거리 변환:")
println("미터: ${distance.value}m")
println("킬로미터: ${distance.toKilometers()}km")
println("피트: %.2f ft".format(distance.toFeet()))

In [ ]:
// 값 클래스를 사용한 타입 안전 ID
@JvmInline
value class UserId(val value: Long)

@JvmInline
value class ProductId(val value: Long)

@JvmInline
value class OrderId(val value: Long)

// 함수 시그니처가 명확해짐
fun findUser(id: UserId): String = "User-${id.value}"
fun findProduct(id: ProductId): String = "Product-${id.value}"
fun findOrder(userId: UserId, orderId: OrderId): String = 
    "Order-${orderId.value} for User-${userId.value}"

// 타입 안전성
val userId = UserId(123)
val productId = ProductId(456)
val orderId = OrderId(789)

println(findUser(userId))
println(findProduct(productId))
println(findOrder(userId, orderId))

// findUser(productId)  // 컴파일 에러! 타입이 맞지 않음

### 💡 핵심 포인트
- **런타임 오버헤드 없음**: 컴파일 시 원시 타입으로 인라인
- **타입 안전성**: 실수로 잘못된 타입 전달 방지
- **도메인 모델링**: 비즈니스 개념을 타입으로 표현
- **단일 프로퍼티만** 가질 수 있음

## 3. 위임 (Delegation)

Kotlin은 클래스 위임과 프로퍼티 위임을 언어 레벨에서 지원합니다.

In [ ]:
// 클래스 위임
interface Printer {
    fun print(message: String)
}

class ConsolePrinter : Printer {
    override fun print(message: String) {
        println("콘솔: $message")
    }
}

// by 키워드로 위임
class PrefixPrinter(
    private val prefix: String,
    private val printer: Printer
) : Printer by printer {
    // 선택적으로 오버라이드 가능
    override fun print(message: String) {
        printer.print("$prefix $message")
    }
}

val printer = PrefixPrinter("[INFO]", ConsolePrinter())
printer.print("애플리케이션 시작")
printer.print("처리 중...")
printer.print("완료!")

In [ ]:
// 프로퍼티 위임 - lazy
class ExpensiveResource {
    init {
        println("무거운 리소스 생성 중...")
        Thread.sleep(1000)  // 시뮬레이션
        println("리소스 생성 완료!")
    }
    
    fun use() = "리소스 사용"
}

class ResourceManager {
    // lazy 위임: 처음 접근할 때 초기화
    val resource by lazy { ExpensiveResource() }
    
    fun doWork() {
        println("작업 시작")
        // 필요할 때만 리소스 생성
        if (Math.random() > 0.5) {
            println(resource.use())
        } else {
            println("리소스 필요 없음")
        }
    }
}

val manager = ResourceManager()
println("매니저 생성됨")
manager.doWork()

In [ ]:
// 커스텀 프로퍼티 위임
import kotlin.reflect.KProperty

class LoggingProperty<T>(private var value: T) {
    operator fun getValue(thisRef: Any?, property: KProperty<*>): T {
        println("${property.name} 읽기: '$value'")
        return value
    }
    
    operator fun setValue(thisRef: Any?, property: KProperty<*>, newValue: T) {
        println("${property.name} 변경: '$value' -> '$newValue'")
        value = newValue
    }
}

class User {
    var name: String by LoggingProperty("")
    var email: String by LoggingProperty("")
}

val user = User()
user.name = "김코틀린"
user.email = "kim@example.com"
println("현재 이름: ${user.name}")
println("현재 이메일: ${user.email}")

In [ ]:
// Observable 프로퍼티
import kotlin.properties.Delegates

class ObservableUser {
    var name: String by Delegates.observable("<초기값>") { prop, old, new ->
        println("${prop.name} 변경됨: $old -> $new")
    }
    
    var age: Int by Delegates.vetoable(0) { prop, old, new ->
        println("${prop.name} 변경 시도: $old -> $new")
        new >= 0  // true면 변경 허용, false면 거부
    }
}

val observableUser = ObservableUser()
observableUser.name = "홍길동"
observableUser.name = "김철수"

println("\n나이 변경 시도:")
observableUser.age = 25
println("현재 나이: ${observableUser.age}")

observableUser.age = -5  // 거부됨
println("현재 나이: ${observableUser.age}")

### 💡 핵심 포인트
- **클래스 위임**: 상속 대신 합성 사용 권장
- **프로퍼티 위임**: 공통 프로퍼티 동작을 재사용
- **lazy**: 지연 초기화
- **observable/vetoable**: 프로퍼티 변경 감지/제어

## 4. 코루틴 (Coroutines)

비동기 프로그래밍을 위한 경량 스레드입니다. async/await 패턴을 지원합니다.

**주의**: Jupyter 환경에서는 코루틴 실행이 제한적일 수 있습니다.

In [ ]:
import kotlinx.coroutines.*

// 기본 코루틴 실행
runBlocking {
    println("코루틴 시작")
    
    // launch: 새 코루틴 시작 (결과 반환 X)
    launch {
        delay(1000)  // 1초 대기 (스레드 블록하지 않음)
        println("launch 코루틴 완료")
    }
    
    // async: 새 코루틴 시작 (결과 반환 O)
    val deferred = async {
        delay(500)
        "async 결과"
    }
    
    println("대기 중...")
    println("async 결과: ${deferred.await()}")
}

In [ ]:
import kotlin.system.measureTimeMillis

// 순차 실행 vs 병렬 실행
runBlocking {
    suspend fun fetchData(name: String, delay: Long): String {
        delay(delay)
        return "$name 데이터"
    }
    
    // 순차 실행
    val time1 = measureTimeMillis {
        val data1 = fetchData("첫 번째", 1000)
        val data2 = fetchData("두 번째", 1000)
        println("순차: $data1, $data2")
    }
    println("순차 실행 시간: ${time1}ms")
    
    // 병렬 실행
    val time2 = measureTimeMillis {
        val deferred1 = async { fetchData("첫 번째", 1000) }
        val deferred2 = async { fetchData("두 번째", 1000) }
        println("병렬: ${deferred1.await()}, ${deferred2.await()}")
    }
    println("병렬 실행 시간: ${time2}ms")
}

In [ ]:
// 코루틴 스코프와 구조적 동시성
runBlocking {
    println("메인 코루틴 시작")
    
    // coroutineScope: 모든 자식이 완료될 때까지 대기
    coroutineScope {
        launch {
            delay(2000)
            println("자식 코루틴 1 완료")
        }
        
        launch {
            delay(1000)
            println("자식 코루틴 2 완료")
        }
        
        println("coroutineScope 내부")
    }
    
    println("모든 자식 코루틴 완료 후 실행")
}

In [ ]:
// Flow - 비동기 데이터 스트림
import kotlinx.coroutines.flow.*

runBlocking {
    // Flow 생성
    fun simpleFlow(): Flow<Int> = flow {
        println("Flow 시작")
        for (i in 1..3) {
            delay(300)  // 비동기 작업 시뮬레이션
            emit(i)     // 값 방출
        }
    }
    
    // Flow 수집
    println("Flow 수집 시작")
    simpleFlow()
        .map { it * it }  // 변환
        .filter { it > 5 } // 필터링
        .collect { value ->
            println("수신: $value")
        }
}

In [ ]:
// 실용적인 예제: 여러 API 호출
runBlocking {
    data class User(val id: Int, val name: String)
    data class Post(val userId: Int, val title: String)
    
    suspend fun fetchUser(id: Int): User {
        delay(1000)  // API 호출 시뮬레이션
        return User(id, "User$id")
    }
    
    suspend fun fetchPosts(userId: Int): List<Post> {
        delay(1500)  // API 호출 시뮬레이션
        return listOf(
            Post(userId, "Post 1"),
            Post(userId, "Post 2")
        )
    }
    
    // 여러 사용자의 정보와 게시물을 병렬로 가져오기
    val userIds = listOf(1, 2, 3)
    
    val time = measureTimeMillis {
        val results = userIds.map { id ->
            async {
                val user = fetchUser(id)
                val posts = fetchPosts(id)
                user to posts
            }
        }.awaitAll()
        
        results.forEach { (user, posts) ->
            println("${user.name}: ${posts.size}개 게시물")
        }
    }
    
    println("총 소요 시간: ${time}ms")
}

### 💡 핵심 포인트
- **경량**: 수천 개의 코루틴도 문제없이 실행
- **구조적 동시성**: 부모-자식 관계로 생명주기 관리
- **취소 가능**: 협력적 취소 지원
- **Flow**: 비동기 데이터 스트림 처리

## 5. DSL (Domain Specific Language)

Kotlin의 문법적 특성을 활용하여 도메인 특화 언어를 만들 수 있습니다.

In [ ]:
// HTML DSL 예제
class HTML {
    private val elements = mutableListOf<String>()
    
    fun head(block: Head.() -> Unit) {
        val head = Head()
        head.block()  // 수신 객체 지정 람다
        elements.add(head.toString())
    }
    
    fun body(block: Body.() -> Unit) {
        val body = Body()
        body.block()
        elements.add(body.toString())
    }
    
    override fun toString() = "<html>\n${elements.joinToString("\n")}\n</html>"
}

class Head {
    private var title = ""
    
    fun title(text: String) {
        title = text
    }
    
    override fun toString() = "  <head>\n    <title>$title</title>\n  </head>"
}

class Body {
    private val content = mutableListOf<String>()
    
    fun h1(text: String) {
        content.add("    <h1>$text</h1>")
    }
    
    fun p(text: String) {
        content.add("    <p>$text</p>")
    }
    
    override fun toString() = "  <body>\n${content.joinToString("\n")}\n  </body>"
}

fun html(block: HTML.() -> Unit): HTML {
    val html = HTML()
    html.block()
    return html
}

// DSL 사용
val document = html {
    head {
        title("Kotlin DSL 예제")
    }
    body {
        h1("DSL로 HTML 생성하기")
        p("이것은 Kotlin DSL로 만든 HTML입니다.")
        p("타입 안전하고 읽기 쉽습니다.")
    }
}

println(document)

In [ ]:
// 설정 DSL 예제
class DatabaseConfig {
    var host = "localhost"
    var port = 5432
    var database = ""
    var username = ""
    var password = ""
    
    fun connectionString() = "jdbc:postgresql://$host:$port/$database"
    
    override fun toString() = """
        Database Configuration:
        - Host: $host:$port
        - Database: $database
        - Username: $username
        - Connection: ${connectionString()}
    """.trimIndent()
}

class ServerConfig {
    var port = 8080
    var contextPath = "/"
    val database = DatabaseConfig()
    
    fun database(block: DatabaseConfig.() -> Unit) {
        database.block()
    }
    
    override fun toString() = """
        Server Configuration:
        - Port: $port
        - Context Path: $contextPath
        
        $database
    """.trimIndent()
}

fun server(block: ServerConfig.() -> Unit): ServerConfig {
    val config = ServerConfig()
    config.block()
    return config
}

// DSL 사용
val config = server {
    port = 9000
    contextPath = "/api"
    
    database {
        host = "db.example.com"
        port = 5433
        database = "myapp"
        username = "admin"
        password = "secret"
    }
}

println(config)

In [ ]:
// 테스트 DSL 예제
class TestCase(val name: String) {
    private val assertions = mutableListOf<() -> Unit>()
    
    fun assert(description: String, block: () -> Boolean) {
        assertions.add {
            val result = block()
            println("  ${if (result) "✓" else "✗"} $description")
            if (!result) {
                throw AssertionError(description)
            }
        }
    }
    
    fun run() {
        println("테스트: $name")
        try {
            assertions.forEach { it() }
            println("  모든 테스트 통과!\n")
        } catch (e: AssertionError) {
            println("  테스트 실패: ${e.message}\n")
        }
    }
}

fun test(name: String, block: TestCase.() -> Unit) {
    val testCase = TestCase(name)
    testCase.block()
    testCase.run()
}

// DSL 사용
test("문자열 확장 함수 테스트") {
    assert("removeSpaces는 공백을 제거한다") {
        "a b c".removeSpaces() == "abc"
    }
    
    assert("isPalindrome은 회문을 검사한다") {
        "level".isPalindrome() == true
    }
    
    assert("isPalindrome은 회문이 아닌 것을 구별한다") {
        "hello".isPalindrome() == false
    }
}

test("정수 확장 함수 테스트") {
    assert("isEven은 짝수를 판별한다") {
        4.isEven() && !5.isEven()
    }
    
    assert("squared는 제곱을 계산한다") {
        5.squared() == 25
    }
}

### 💡 핵심 포인트
- **수신 객체 지정 람다**: `block: T.() -> Unit`
- **중위 함수**: 자연스러운 문법
- **연산자 오버로딩**: 직관적인 API
- **타입 안전**: 컴파일 타임 검증

## 실습 과제

고급 기능들을 활용하여 간단한 의존성 주입(DI) 프레임워크를 구현해보세요.

In [ ]:
// 과제: 미니 DI 프레임워크
// TODO: 다음 기능을 구현하세요
// 1. @Inject 어노테이션
// 2. Container 클래스 (의존성 등록/해결)
// 3. DSL로 의존성 설정
// 4. 지연 초기화 지원

// 예시 사용법:
// val container = container {
//     singleton<Database> { MySQLDatabase() }
//     factory<Service> { UserService(get()) }
// }
// 
// val service: Service = container.get()

// 여기에 코드를 작성하세요
